In [1]:
import sys
sys.path.insert(0, '../src/core/')

from llm import ILLM, LLMBuilder
illm: ILLM = LLMBuilder(llm_type='ollama', model_name='llama3', temperature=0)

In [2]:
import datetime

from langchain.agents import Tool                                               # type: ignore
from langchain.agents import OpenAIMultiFunctionsAgent                          # type: ignore
from langchain.agents import initialize_agent, AgentType                        # type: ignore
from langchain.prompts import MessagesPlaceholder                               # type: ignore
from langchain.schema.messages import SystemMessage                             # type: ignore
from langchain.chains.conversation.memory import ConversationBufferMemory       # type: ignore


memory = ConversationBufferMemory(return_messages=True, k=10, memory_key="chat_history")

# Você pode criar a ferramenta para passar para um agente
tools = [
    Tool(name="LLM", func=illm.get_instance().invoke, description="Performs language model-based operations")
]

# Ajuste da mensagem do sistema
prompt = OpenAIMultiFunctionsAgent.create_prompt(
    system_message=SystemMessage(content="You are a helpful AI bot"),
    extra_prompt_messages=[MessagesPlaceholder(variable_name="chat_history")],
)

# Inicializar o agente
agent_executor = initialize_agent(
    tools,
    illm.get_instance(),
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=10,
    memory=memory,
    prompt=prompt,
)

# Exemplo de uso
response = agent_executor.run({ "chat_history": [], "input": "Ola, tudo bem?" })

print("Agent Response:", response)


/home/diegoomal/miniconda3/envs/python-flask-chat-env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/home/diegoomal/miniconda3/envs/python-flask-chat-env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Action:
{
  "action": "LLM",
  "action_input": {"type": "string"}
}



> Finished chain.
Agent Response: Action:
{
  "action": "LLM",
  "action_input": {"type": "string"}
}




In [3]:
print(memory.buffer)

[HumanMessage(content='Ola, tudo bem?'), AIMessage(content='Action:\n{\n  "action": "LLM",\n  "action_input": {"type": "string"}\n}\n\n')]


In [4]:
print(agent_executor.agent.llm_chain.prompt)

input_variables=['agent_scratchpad', 'input'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nLLM: Performs language model-based operations, args: {{\'tool_input\': {{\'type\': \'string\'}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or LLM\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n```\n\nBegin! Reminder to A